In [10]:
from datetime import datetime, timedelta

datetime.now(tz="Asia/Seoul") - timedelta(hours=24)


datetime.datetime(2024, 7, 15, 16, 52, 51, 206073)

In [9]:
import requests
from enum import Enum

class TaskStatus(Enum):
    RUNNING = 'RUNNING'
    DONE = 'DONE'
    FAILED = 'FAILED'
url = 'http://localhost:8080/api/taskruns/1/status'
headers = {'Content-Type': 'application/json'}
response = requests.post(url=url, json={'status': TaskStatus.DONE.value})

print(f"Status Code: {response.status_code}")
print(f"Response Body: {response.text}")

Status Code: 200
Response Body: {"id":1,"taskName":"엔비디아 Task","taskDays":1,"status":"DONE","createdAt":"2024-07-13T07:29:58.094248Z","finishedAt":"2024-07-13T16:37:32.183138+09:00"}


In [1]:
import os
from dotenv import load_dotenv
from tools.crawlers import NaverApiCrawler

load_dotenv()
NAVER_CLIENT_ID = os.getenv('NAVER_CLIENT_ID')
NAVER_CLIENT_SECRET = os.getenv('NAVER_CLIENT_SECRET')
NAVER_API_ENDPOINT = os.getenv('NAVER_API_ENDPOINT')
DAYS = int(os.getenv("DAYS"))
SELENIUM_TIMEOUT_SECONDS = int(os.getenv('SELENIUM_TIMEOUT_SECONDS'))

keywords = ["테슬라"]

api_crawler = NaverApiCrawler(NAVER_CLIENT_ID, NAVER_CLIENT_SECRET, NAVER_API_ENDPOINT)

articles_by_keyword = {}
for keyword in keywords:
    articles_by_keyword[keyword] = api_crawler.get_articles(keyword, DAYS)

for keyword, value in articles_by_keyword.items():
    print("Keyword: {}\tCount: {}".format(keyword, len(value)))

Keyword: 테슬라	Count: 126


In [2]:
from tools.crawlers import WebCrawler

content_crawler = WebCrawler()

for keyword, docs in articles_by_keyword.items():
    for doc in docs:
        doc['title'], doc['description'] = content_crawler.crawl_naver_news(doc['link'], SELENIUM_TIMEOUT_SECONDS)

In [3]:
import datetime
from elasticsearch import Elasticsearch, helpers

ELASTICSEARCH_ENDPOINT = os.getenv("ELASTICSEARCH_ENDPOINT")
ELASTICSEARCH_USER = os.getenv("ELASTICSEARCH_USER")
ELASTICSEARCH_PWD = os.getenv("ELASTICSEARCH_PWD")
ELASTICSEARCH_INDEX_PREFIX = os.getenv("ELASTICSEARCH_INDEX_PREFIX")


es = Elasticsearch(
    ELASTICSEARCH_ENDPOINT,
    basic_auth=(ELASTICSEARCH_USER, ELASTICSEARCH_PWD)
)

data = []
for keyword, articles in articles_by_keyword.items():
    for article in articles:
        index_suffix = article['pubDate'].strftime("%Y.%m.%d")
        index_name = "".join([ELASTICSEARCH_INDEX_PREFIX, index_suffix])
        article_id = article["id"]
        del article['id']
        doc = {
            "_index": index_name,
            "_id": article_id,
            "_source": article
        }
        data.append(doc)

success, failed = helpers.bulk(es, data)
print(success, failed)


126 []


In [4]:

data[0]


{'_index': 'articles-2024.07.07',
 '_id': '3660001002700',
 '_source': {'title': '진격의 테슬라…서학개미 주식 보관액, 엔비디아 제쳤다',
  'originallink': 'https://biz.chosun.com/stock/stock_general/2024/07/07/FBCCANDNMJA75F7HODA76HZOJI/?utm_source=naver&utm_medium=original&utm_campaign=biz',
  'link': 'https://n.news.naver.com/mnews/article/366/0001002700?sid=101',
  'description': '일론 머스크 테슬라 CEO. /AP연합뉴스\n\n테슬라에 베팅한 서학개미(해외 주식에 투자하는 국내 개인 투자자)들이 오랜만에 다시 웃게 됐다. 개인 투자자들의 테슬라 주식 보관액(투자자들이 보유 중인 주식의 평가액)이 3주 만에 엔비디아를 제쳤기 때문이다. 최근 테슬라 주가가 급등하며 250달러를 넘었기 때문인데, 월스트리트에서는 목표주가를 앞다퉈 300달러대로 상향 조정하고 있어 관심이 쏠린다.\n\n7일 한국예탁결제원 증권정보 포털 ‘세이브로’에 따르면, 지난 4일 해외 주식 가운데 보관액이 가장 큰 종목은 테슬라였다. 총 146억6985만달러(약 20조2800억원)어치를 보유 중인 것으로 나타났다. 2위는 엔비디아(134억달러·약 18조5500억원), 3위는 애플(49억달러·약 6조8700억원)이었다.\n\n테슬라 주식 보관액이 엔비디아를 추월한 것은 지난 2일부터다. 6월 11일 이후 3주 만이었다. 테슬라 주가가 지난달 말부터 급등하고 있는 반면 엔비디아는 박스권에 머물고 있기 때문이다.\n\n테슬라 주가는 최근 한 달 간 41% 상승한 것으로 나타났다. 올해 4월 142달러 수준까지 떨어지는 굴욕을 겪기도 했지만, 현재(5일 기준) 251.52달러까지 오른 상태다. 테슬라 주가가 250달러를 넘은 건 작년 1

In [6]:
os.uname().nodename

'idongjuui-MacBookPro.local'

In [8]:
print(os.getenv("HOSTNAME"))

None
